# Last step
# Predict segmentations

In [1]:
%matplotlib inline

In [2]:
import os
import os.path

import glob

import matplotlib.pyplot as plt
import numpy as np

import skimage.io
import skimage.morphology

from tqdm import tqdm

import tensorflow as tf
import keras

import utils.metrics
import utils.model_builder

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Configurable parameter:
* config (experiment)
* GPU
* input folder 
* output folder

# Configuration

In [7]:
GPU_NO = "0"
input_directory =  "/storage/data/2018_tim_tracking/2018_08_06_migration_perturbations_asthma/images/test_sequence/migration_006_HEM_none_IL8_100ngml_1/*jpg"
output_directory = "/storage/data/2018_tim_tracking/2018_08_06_migration_perturbations_asthma/images/test_sequence/migration_006_HEM_none_IL8_100ngml_1_prediction_neutros_v3/"
experiment_name = 'neutros_v3'


In [9]:
from config import config_vars

# load configuration file
config_vars = utils.dirtools.setup_experiment(config_vars, experiment_name)

# output directory
config_vars["probmap_out_dir"] = output_directory

# initialize GPU
configuration = tf.ConfigProto()
configuration.gpu_options.allow_growth = True
configuration.gpu_options.visible_device_list = GPU_NO

session = tf.Session(config = configuration)

# apply session
keras.backend.set_session(session)


# Load and preprocessing images

In [5]:
image_names = glob.glob(input_directory)
images = []
imagebuffer = skimage.io.imread_collection( image_names)
for image in imagebuffer:
    #image = skimage.io.imread(filename)
    images.append(skimage.color.rgb2gray(image))
    
print("found {} images".format(len(imagebuffer)))

images = np.array(images)

dim1 = np.floor(images.shape[1]/16) * 16 
dim1 = dim1.astype(np.int)
dim2 = np.floor(images.shape[2]/16) * 16 
dim2 = dim2.astype(np.int)

images = images[:,0:dim1,0:dim2]

dim1 = images.shape[1]
dim2 = images.shape[2]

images = images.reshape((-1, dim1, dim2, 1))

print(dim1,dim2)

# preprocess images
percentile = 99.9
for image_no in tqdm(range(images.shape[0])):
    orig_img = images[image_no,:,:,:]
    
    high = np.percentile(orig_img, percentile)
    low = np.percentile(orig_img, 100-percentile)

    img = np.minimum(high, orig_img)
    img = np.maximum(low, img)
    img = (img - low) / (high - low) 
    img = skimage.img_as_ubyte(img) 
    images[image_no,:,:,:] = img # gives float64, thus cast to 8 bit later
    

  
images = images.astype(float)
images = images / 256

# plt.imshow(images[1,1:500, 1:500,:].mean(axis=2))
# plt.show()
#
# print(images.max())
# print(images.dtype)


found 207 images


  0%|          | 0/207 [00:00<?, ?it/s]/home/0x00b1/com/github/scikit-image/scikit-image/skimage/util/dtype.py:123: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
  2%|▏         | 4/207 [00:00<00:06, 32.24it/s]

720 1280


100%|██████████| 207/207 [00:07<00:00, 27.55it/s]


# Predict images 

In [6]:
# build model and load weights
model = utils.model_builder.get_model_3_class(dim1, dim2)
model.load_weights(config_vars["model_file"])

#  prediction 
predictions = model.predict(images, batch_size=1)


/home/cells2numbers/2018_unet_segmentation_hudy/unet4nuclei/unet4nuclei/utils/model_builder.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
  a = keras.layers.Convolution2D(64, 3, 3, **option_dict_conv)(x)
/home/cells2numbers/2018_unet_segmentation_hudy/unet4nuclei/unet4nuclei/utils/model_builder.py:17: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(momentum=0.9)`
  a = keras.layers.BatchNormalization(**option_dict_bn)(a)
/home/cells2numbers/2018_unet_segmentation_hudy/unet4nuclei/unet4nuclei/utils/model_builder.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
  a = keras.layers.Convolution2D(64, 3, 3, **option_dict_conv)(a)
/home/cells2numbers/2018_unet_segmentation_hudy/unet4nuclei/unet4nuclei/utils/model_builder.py:20: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `Batch

series_0029_frame_0110.png


/home/0x00b1/com/github/scikit-image/scikit-image/skimage/util/dtype.py:123: UserWarning: Possible precision loss when converting from float32 to uint8
  .format(dtypeobj_in, dtypeobj_out))


FileNotFoundError: [Errno 2] No such file or directory: '/storage/data/2018_tim_tracking/2018_08_06_migration_perturbations_asthma/images/test_sequence/migration_006_HEM_none_IL8_100ngml_1_prediction_neuitros_v3/series_0029_frame_0110.png'

In [10]:

for i in range(len(images)):

    filename = imagebuffer.files[i]
    filename = os.path.basename(filename)
    filename = os.path.splitext(filename)[0] + ".png"
    print(filename)
    
    probmap = predictions[i].squeeze()
    
    skimage.io.imsave(config_vars["probmap_out_dir"] + filename, probmap)
    
    #pred = utils.metrics.probmap_to_pred(probmap, config_vars["boundary_boost_factor"])

    #plt.imshow(pred)
    #plt.show()
    
    #label = utils.metrics.pred_to_label(pred, config_vars["cell_min_size"])
    
    #plt.imshow(label)
    #plt.show()
    
    #skimage.io.imsave(config_vars["labels_out_dir"] + filename, label)

series_0029_frame_0110.png
series_0029_frame_0123.png


/home/0x00b1/com/github/scikit-image/scikit-image/skimage/util/dtype.py:123: UserWarning: Possible precision loss when converting from float32 to uint8
  .format(dtypeobj_in, dtypeobj_out))


series_0029_frame_0120.png
series_0029_frame_0151.png
series_0029_frame_0084.png
series_0029_frame_0174.png
series_0029_frame_0100.png
series_0029_frame_0126.png
series_0029_frame_0007.png
series_0029_frame_0203.png
series_0029_frame_0138.png
series_0029_frame_0185.png
series_0029_frame_0199.png
series_0029_frame_0045.png
series_0029_frame_0193.png
series_0029_frame_0159.png
series_0029_frame_0093.png
series_0029_frame_0019.png
series_0029_frame_0014.png
series_0029_frame_0098.png
series_0029_frame_0107.png
series_0029_frame_0065.png
series_0029_frame_0103.png
series_0029_frame_0146.png
series_0029_frame_0076.png
series_0029_frame_0028.png
series_0029_frame_0127.png
series_0029_frame_0116.png
series_0029_frame_0161.png
series_0029_frame_0119.png
series_0029_frame_0150.png
series_0029_frame_0135.png
series_0029_frame_0206.png
series_0029_frame_0003.png
series_0029_frame_0008.png
series_0029_frame_0177.png
series_0029_frame_0158.png
series_0029_frame_0121.png
series_0029_frame_0043.png
s